<a href="https://colab.research.google.com/github/Purushotham011/Gen-AI/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers,Model
from tensorflow.keras import datasets
from tensorflow.keras.losses import MeanSquaredError, KLDivergence, binary_focal_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from numpy import cov, trace, iscomplexobj,asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from skimage.transform import resize

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

In [5]:
latent_dim = 1024
image_shape = x_train[0].shape
mse_loss = MeanSquaredError()
kl_loss = KLDivergence()

In [6]:
class VAE(keras.Model) :
    def __init__(self,latent_dim,image_shape,beta) :
        super(VAE,self).__init__()
        self.latent_dim = latent_dim
        self.image_shape = image_shape
        self.beta = beta

        #encoder
        self.encoder = keras.Sequential([
            layers.InputLayer(shape=image_shape),name ="Encoder_Input_Layer",

            layers.Conv2D(filters=32,kernel_size=3,strides=(2,2),activation='relu',padding='same',name="Encoder_Conv2D_Layer_2"),

            layers.Conv2D(filters=64,kernel_size=3,strides=(2,2),activation='relu',padding='same',name="Encoder_Conv2D_Layer_4"),

            layers.Conv2D(filters=128,kernel_size=3,strides=(2,2),activation='relu',padding='same',name="Encoder_Conv2D_Layer_6"),

            layers.Flatten(name="Encoder_Flatten"),
        ])

        self.z_mean = layers.Dense(latent_dim,name="z_mean")
        self.z_log_var = layers.Dense(latent_dim,name="z_log_var")

        #decoder
        self.decoder = keras.Sequential([
            layers.InputLayer(shape=(latent_dim,)),
            layers.Dense(units=8*8*64, activation='relu'),
            layers.Reshape(target_shape=(8, 8, 64)),
            layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding='same', activation='relu'),
            layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding='same', activation='relu'),
            layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding='same' , activation='sigmoid')
        ])

        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    #Build the Encoder
    def encode(self,data):
        x= slef.encoder(data)
        z_mean,z_log_var = self.z_mean(x),self.z_log_var(x)
        return z_mean,z_log_var

    def reparameterize(self,z_mean,z_log_var):
        batch =tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch,dim))
        z = z_mean + tf.exp(0.5*z_log_var)*epsilon
        return z

    def call(self,x):
        mean, logvar =self.encode
        z = self.reparameterize(mean,logvar)
        return self.decoder(z),mean,logvar
